In [3]:
import os
import torch
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.tongyi import Tongyi
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.passthrough import RunnablePassthrough

# 加载嵌入模型
embedding_model = HuggingFaceEmbeddings(
    model_name="/Users/zhangyf/llm/bge-base-zh-v1.5",
    model_kwargs={"device": "mps" if torch.mps.is_available() else "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

# 初始化 Chroma 客户端
vectorstore = Chroma(
    persist_directory="vectorstore",
    embedding_function=embedding_model,
)

# 创建检索器
retriever = vectorstore.as_retriever(
    search_type="similarity",  # 检索方式，similarity 或 mmr
    search_kwargs={"k": 3},
)


In [7]:
# 检索与生成链条
load_dotenv()
TONGYI_API_KEY = os.getenv("TONGYI_API_KEY")
# 大模型
llm = Tongyi(model="qwen-turbo", api_key=TONGYI_API_KEY)
# 本地模型


# 将检索到的文档中的 page_content 取出组合到一起
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Prompt 模板
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template="""
你是一个专业的中文问答助手，擅长基于提供的资料回答用户问题。
请仅根据以下背景资料回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：{context}

问题：{query}

回答：""",
)



# RAG 链条
rag_chain = (
    {"context": retriever | format_docs, "query": RunnablePassthrough()}
    | prompt
    | (lambda x: print(x.text, end="") or x)
    | llm
    | StrOutputParser()  # 输出解析器，将输出解析为字符串
)

# query = "不动产或者动产被占有人占有怎么办"
query = "今天天气怎么样？"
response = rag_chain.invoke(query)
print(response)



你是一个专业的中文问答助手，擅长基于提供的资料回答用户问题。
请仅根据以下背景资料回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：(一 ) 政府 采购 情况 说 明

2023 年 是 深入 推 进 实施 “十 四 五 ?战略 规划 、 五 规划 目标 实现 的 关键 之 年 , 抢 抓 机 遇 争 取 承 担 国 推动 * 十 家 重大 科 四 技 任务 ， 管 理 运行 好 重大 观测 设施 ， 组 织 做 好 各 类 科学 数据 处 理 的 应 用 研究 工作 ， 努 力 产 出 更 多 更 大 科学 成 果 。

2023 年 工作 总 体 思路 是 : 深入 学 习 贯 彻 党 的 二 十 大 精 神 ， 沙 实 党 的 全 面 领导 ， 落 实 党 中 央 、 国 务 院 和 院 党 组 重 决策 部 署 ， 面 向 国家 重大 需求 、 面 向 国际 天 文科 技 前 沿 、 面 向 经 济 主 战场 ， 加 快 j 全 进 国 家 “十 四 五 ?科技 发 展 规划 实施 ， 加 快 建设 和 持续 推进 国家 重点 实验 室 重 组 工作 ， 加 快 协 同 凝 聚 天 文 领域 科研 队伍 ， 加 强 青年 人 才 队 伍 建设 ， 加 质量 发 展 新 阶段 ， 为 创新 型 国 ATK. 更 REN 家 和 世界 科技 强国 建设 做 出

问题：今天天气怎么样？

回答：我不知道
